**python -m venv ./venv**

Run that in your terminal to set up the virtual environment, then run the below command. If you don't care about using a virtual environment, just run the below command to install the required packages for your current version of Python.

In [ ]:
%pip install pandas numpy matplotlib seaborn scikit-learn tensorflow wordcloud

You can also run the following:

In [ ]:
%pip install -r requirements.txt

**Process 1: Data Cleaning**

In [ ]:
#import data
import pandas as pd
import numpy as np

data = pd.read_csv('Flight Price Prediction Data/Clean_Dataset.csv')
data.head()

In [ ]:
#drop 1st column
data = data.drop(data.columns[0], axis=1)
data.head()

In [ ]:
#CREATE NEW COLUMN TO CONVERT PRICES TO USD

#conversion rate from INR to USD
conversion_rate = 0.012

#function to convert price from INR to USD
def convert_to_usd(price_inr):
    return price_inr * conversion_rate

#create new column for USD price, applying conversion and round to the nearest cent
data['priceUSD'] = data['price'].apply(convert_to_usd).apply(lambda x: round(x, 2))

#rename current price column to indicate it is INR (Indian Rupee)
data = data.rename(columns={'price': 'priceINR'})

data.head()

In [ ]:
#CONVERT STOPS VALUES FROM STRING TO NUMERICAL

#mapping dictionary
number_mapping = {'zero': 0, 'one': 1, 'two_or_more': 2}

# Convert string versions of numbers to numerical values
data['stops'] = data['stops'].map(number_mapping)

data.head()